In [169]:
# import statements
import requests
import re
import json
import time
from bs4 import BeautifulSoup as BS

In [170]:
# scrape wikipedia page
def scrape(url):
    # need header to scrape from wikipedia
    headers = {
        'User-Agent': 'SchoolProjectBot/1.0 (r1043412@student.thomasmore.be) Python-Requests/2.31.0'
    }
    page = requests.get(url, headers=headers)
    
    return BS(page.text, "html")

In [ ]:
def yellow_jersey(soup):
    table = soup.find_all('table', class_ = 'wikitable')

    # debug 
    print(table[1].select('tr')[0])

    # use list comprehension instead of for loop
    riders = [
        # take all td's and put them in a list 'cells', take the second and third elements from that list
        # is the lambda TRULY needed here TODO
        (lambda cells: (cells[1].get_text(strip=True), cells[4].get_text(strip=True)))(row.find_all("td"))

        # take all rows exept title and results row
        for row in table[1].select('tr')[1:]
    ]

    riders_dict = [
        {'name': rider[0], 'jerseys': rider[1]}
        for rider in riders
    ]

    return riders_dict

In [172]:
def white_jersey(soup):
    table = soup.find('table', class_ = 'wikitable')

    # use list comprehension instead of for loop
    riders = [
        # take all td's and put them in a list 'cells', take the second and third elements from that list
        (lambda cells: cells[1].get_text(strip=True))(row.find_all("td"))

        # take all rows exept title and results row
        for row in table.select('tr')[1:]
    ]

    riders_dict = dict()
    for rider in riders:
        if rider not in riders_dict:
            riders_dict.update({rider: 1})
        else:
            riders_dict[rider] += 1

    return riders_dict

In [173]:
def dotted_jersey(soup):
    table = soup.find_all('table', class_ = 'wikitable')

    # use list comprehension instead of for loop
    riders = [
        # take all td's and put them in a list 'cells', take the second and third elements from that list
        (lambda cells: cells.get_text(strip=True))(row.find("th"))

        # take all rows exept title and results row
        for row in table[3].select('tr')[1:]
    ]

    riders_dict = dict()
    for rider in riders:
        if rider not in riders_dict:
            riders_dict.update({rider: 1})
        else:
            riders_dict[rider] += 1

    return riders_dict

In [174]:
def green_jersey(soup):
    table = soup.find_all('table', class_ = 'wikitable')

    riders = [
        # take all td's and put them in a list 'cells', take the second and third elements from that list
        (lambda cells: cells[1].get_text(strip=True))(row.find_all("td"))

        # take all rows exept title and results row
        for row in table[3].select('tr')[1:]
    ]

    riders_dict = dict()
    for rider in riders:
        if rider not in riders_dict:
            riders_dict.update({rider: 1})
        else:
            riders_dict[rider] += 1

    return riders_dict

In [175]:
def write_json(dict, type):
    with open(f'data/jerseys.json', 'r+') as file:
        # load the entire json file into memory
        file_json = json.load(file)

        # update the json with the python dictionary
        file_json[type].append(dict)

        # put cursor at beginning of file and overwrite it with the dictionary in memory
        file.seek(0)
        json.dump(file_json, file, indent=4)

In [ ]:
urls = [
    'https://en.wikipedia.org/wiki/Yellow_jersey_statistics',
    'https://en.wikipedia.org/wiki/Young_rider_classification_in_the_Tour_de_France',
    'https://en.wikipedia.org/wiki/Mountains_classification_in_the_Tour_de_France',
    'https://en.wikipedia.org/wiki/Points_classification_in_the_Tour_de_France'
]

jersey_soup = scrape(urls[0])
jersey_dict = yellow_jersey(jersey_soup)
write_json(jersey_dict, 'yellow')
time.sleep(1)

jersey_soup = scrape(urls[1])
jersey_dict = white_jersey(jersey_soup)
write_json(jersey_dict, 'white')
time.sleep(1)

jersey_soup = scrape(urls[2])
jersey_dict = dotted_jersey(jersey_soup)
write_json(jersey_dict, 'dotted')
time.sleep(1)

jersey_soup = scrape(urls[3])
jersey_dict = green_jersey(jersey_soup)
write_json(jersey_dict, 'green')